# ASSIGNMENT 1 - EXCERCISE 2

### Solve a travelling salesman problem for 10 random matrices

#### The following example will be shown with full detail for a 14x14 matrix. Since the assignment requires to solve 10 different matrices, only the final result will be shown for the remaining 9 tests

In [1]:
from pulp import *
import numpy as np

In [2]:
import random
def matrix(n):
    m = [[0] * n for _ in range(n)]
    for row in range(n):
        for column in range(row, n):
            m[column][row] = m[row][column] = 0 if row == column else random.randint(0,100)
    return m
origin_destination_matrix = matrix(14)
print(origin_destination_matrix)

[[0, 6, 60, 34, 98, 34, 45, 87, 24, 23, 27, 34, 78, 98], [6, 0, 52, 48, 51, 74, 97, 71, 31, 33, 31, 52, 10, 56], [60, 52, 0, 92, 28, 82, 19, 80, 24, 34, 67, 16, 18, 39], [34, 48, 92, 0, 77, 19, 52, 19, 60, 24, 64, 45, 44, 60], [98, 51, 28, 77, 0, 99, 33, 11, 30, 56, 68, 53, 15, 3], [34, 74, 82, 19, 99, 0, 56, 53, 47, 1, 76, 64, 32, 22], [45, 97, 19, 52, 33, 56, 0, 63, 91, 82, 59, 95, 79, 73], [87, 71, 80, 19, 11, 53, 63, 0, 87, 42, 59, 80, 37, 51], [24, 31, 24, 60, 30, 47, 91, 87, 0, 66, 27, 86, 43, 85], [23, 33, 34, 24, 56, 1, 82, 42, 66, 0, 63, 51, 69, 61], [27, 31, 67, 64, 68, 76, 59, 59, 27, 63, 0, 93, 11, 28], [34, 52, 16, 45, 53, 64, 95, 80, 86, 51, 93, 0, 46, 70], [78, 10, 18, 44, 15, 32, 79, 37, 43, 69, 11, 46, 0, 17], [98, 56, 39, 60, 3, 22, 73, 51, 85, 61, 28, 70, 17, 0]]


In [3]:
#Create an origin-destination matrix of the distances between the cities
#Note that the center diagonal is of 0's since the pair is origin i = destination j
#First instance will be created with 14 cities
import random
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
#origin_destination_matrix = (arrange(list(range(14*14)), 14, 14))
#print(origin_destination_matrix)

In [4]:
#Create variables with the parameters of the matrix to work with
cities = len(origin_destination_matrix)
indexes = range(cities)
print("Cities: ", cities)
print("Indexes: ", indexes)

Cities:  14
Indexes:  range(0, 14)


In [5]:
#Create a list of all the coordinates combinations of the matrix, excluding the center diagonal of 0's
E = [(i,j) for i in indexes for j in indexes if i!=j]

In [6]:
#Instruction for minimizing the problem
m = LpProblem("TSP",LpMinimize)

In [7]:
#Create a dictionary of all coordinate pairs
#Associate each pair with an X to assign 0 or 1 afterwards if the city will be chosen as part of the optimal route
x = LpVariable.dicts('x',E,cat=LpBinary)

In [8]:
#Create the objective function of costs per each pair
#The costs are the distances stated in the origin-destination matrix, excluding the diagonal
m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])

In [9]:
#Create the constraints
#It limits to a step size of 1, where from one origin, movement can only occur to one destination at a time
#The function calculates each pair both back and forth 
for i in indexes:
    m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
    m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)

In [10]:
#Instructions for preliminar solution of the system without adding the restriction of subtours
m.solve()
vobj = value(m.objective)
xsol = {e:value(x[e]) for e in E}
print("- Optimal routes without eliminating subtours:")
sol_summary={x for x,y in xsol.items() if y!=0}
print("Segments within route: ", sol_summary)

- Optimal routes without eliminating subtours:
Segments within route:  {(1, 12), (6, 2), (12, 1), (13, 4), (4, 13), (10, 8), (11, 0), (3, 7), (8, 10), (7, 3), (9, 5), (2, 6), (5, 9), (0, 11)}


In [11]:
#Find successive cities that have the shortest path between them
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt

In [12]:
#Iterate the previous definition to find all possible short subtours
#The function will add more nodes as it runs out of shorter routes
while True:
    subt = subtour(xsol)
    if len(subt) == cities:
        print("- Optimal tour excluding subtours")
        print("Optimal tour found: %r"%subt)
        print("Optimal tour length: %g"%vobj)
        break
    print("- Identify subtours")
    print("Subtour found: %r"%subt)
    m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
    m.solve()
    vobj = value(m.objective)
    xsol = {e: value(x[e]) for e in E}

- Identify subtours
Subtour found: [0, 11]
- Identify subtours
Subtour found: [0, 1, 12, 13, 4, 6, 2, 11]
- Identify subtours
Subtour found: [0, 11, 2, 6, 4, 13, 12, 10, 8, 1]
- Identify subtours
Subtour found: [0, 11, 3, 7, 4, 13, 12, 1]
- Identify subtours
Subtour found: [0, 1, 12, 13, 4, 7, 3, 6, 2, 11]
- Identify subtours
Subtour found: [0, 1, 8, 10, 12, 13, 4, 7, 3, 11]
- Identify subtours
Subtour found: [0, 8, 10, 12, 1]
- Identify subtours
Subtour found: [0, 1, 8, 10, 12, 13, 4, 7, 3, 6, 2, 11]
- Identify subtours
Subtour found: [0, 9, 5, 13, 4, 6, 2, 11]
- Identify subtours
Subtour found: [0, 11, 9, 5, 3, 7, 4, 13, 12, 1]
- Identify subtours
Subtour found: [0, 11, 2, 6, 4, 7, 3, 9, 5, 13, 12, 1]
- Identify subtours
Subtour found: [0, 1, 8, 10, 12, 13, 4, 7, 3, 5, 9, 11]
- Optimal tour excluding subtours
Optimal tour found: [0, 11, 2, 6, 4, 7, 3, 9, 5, 13, 12, 10, 8, 1]
Optimal tour length: 271


#### Solution for matrices from 5x5 to 13x13

In [13]:
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt
for index in range(5,14):
    print("===================================================================================================================")
    print("Matrix = ", index, "x", index)
    origin_destination_matrix = matrix(index)
    cities = len(origin_destination_matrix)
    indexes = range(cities)
    E = [(i,j) for i in indexes for j in indexes if i!=j]
    m = LpProblem("TSP",LpMinimize)
    x = LpVariable.dicts('x',E,cat=LpBinary)
    m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])
    for i in indexes:
        m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
        m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)
    m.solve()
    vobj = value(m.objective)
    xsol = {e:value(x[e]) for e in E}
    print("- Optimal routes without eliminating subtours:")
    sol_summary={x for x,y in xsol.items() if y!=0}
    print("Segments within route: ", sol_summary)

    while True:
        subt = subtour(xsol)
        if len(subt) == cities:
            print("- Optimal tour excluding subtours")
            print("Optimal tour found: %r"%subt)
            print("Optimal tour length: %g"%vobj)
            break
        print("- Identify subtours")
        print("Subtour found: %r"%subt)
        m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
        m.solve()
        vobj = value(m.objective)
        xsol = {e: value(x[e]) for e in E}

Matrix =  5 x 5
- Optimal routes without eliminating subtours:
Segments within route:  {(0, 1), (4, 0), (1, 2), (3, 4), (2, 3)}
- Optimal tour excluding subtours
Optimal tour found: [0, 1, 2, 3, 4]
Optimal tour length: 74
Matrix =  6 x 6
- Optimal routes without eliminating subtours:
Segments within route:  {(2, 0), (1, 4), (0, 2), (5, 3), (4, 1), (3, 5)}
- Identify subtours
Subtour found: [0, 2]
- Identify subtours
Subtour found: [0, 2, 4, 1]
- Optimal tour excluding subtours
Optimal tour found: [0, 4, 1, 5, 3, 2]
Optimal tour length: 244
Matrix =  7 x 7
- Optimal routes without eliminating subtours:
Segments within route:  {(1, 2), (0, 4), (2, 1), (4, 5), (5, 0), (3, 6), (6, 3)}
- Identify subtours
Subtour found: [0, 4, 5]
- Identify subtours
Subtour found: [0, 5]
- Identify subtours
Subtour found: [0, 5, 4, 6, 3]
- Identify subtours
Subtour found: [0, 2, 1, 4, 5]
- Optimal tour excluding subtours
Optimal tour found: [0, 5, 4, 6, 3, 1, 2]
Optimal tour length: 163
Matrix =  8 x 8
- Op